# evaluate the cfs generated from GPT in IMDB

In [4]:
%load_ext autoreload
%autoreload 2

## finetune the distilBert on IMDB for text sentiment classification

In [5]:
# load distilbert and finetune on IMDB with cuda
from transformers import AutoModel
import torch
model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
## load tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [7]:
## load dataset
from datasets import load_dataset
imdb = load_dataset("imdb")
imdb

Found cached dataset imdb (/home/xiaoqi/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [8]:
unqiue_label_test = set(imdb['test'][:]['label'])
unqiue_label_test

{0, 1}

In [9]:
unqiue_label_unsupervised = set(imdb['unsupervised'][:]['label'])
unqiue_label_unsupervised

{-1}

In [10]:
## preprocess the dataset
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)
# tokenize the imdb dataset
imdb_encoded = imdb.map(tokenize, batched=True, batch_size=None)

Loading cached processed dataset at /home/xiaoqi/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-29fda4695c7227c6.arrow


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Loading cached processed dataset at /home/xiaoqi/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-92955a9b414c79da.arrow


In [11]:
print('map result:')
print(imdb_encoded['train'][:1].keys())

map result:
dict_keys(['text', 'label', 'input_ids', 'attention_mask'])


In [12]:
## load pre_trained model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}
labels_num=2
model = (AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=labels_num, id2label=id2label, label2id=label2id
).to(device))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
## define metric
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # get the prediction index 0 / 1
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
# train

from transformers import Trainer, TrainingArguments
batch_size = 8
logging_steps = len(imdb_encoded["train"]) // batch_size

print(logging_steps)
model_name = f"{model_ckpt}-finetuned-imdb"
training_args = TrainingArguments(output_dir=model_name,
                                num_train_epochs=2,
                                learning_rate=2e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                evaluation_strategy="steps",
                                disable_tqdm=False,
                                logging_steps=logging_steps,
                                push_to_hub=False,
                                log_level="error")

trainer = Trainer(model=model, args=training_args,
                compute_metrics=compute_metrics,
                train_dataset=imdb_encoded["train"],
                eval_dataset=imdb_encoded["test"],
                tokenizer=tokenizer)

trainer.train()                              

3125


/home/xiaoqi/anaconda3/envs/polyjuice-gpupy38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
3125,0.205400,0.233889,0.931360


/home/xiaoqi/anaconda3/envs/polyjuice-gpupy38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/xiaoqi/anaconda3/envs/polyjuice-gpupy38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/xiaoqi/anaconda3/envs/polyjuice-gpupy38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/xiaoqi/anaconda3/envs/polyjuice-gpupy38/lib/python3.8/site-packages/to

TrainOutput(global_step=3126, training_loss=0.20543054214327747, metrics={'train_runtime': 534.1339, 'train_samples_per_second': 93.609, 'train_steps_per_second': 5.852, 'total_flos': 6623369932800000.0, 'train_loss': 0.20543054214327747, 'epoch': 2.0})

## test to predict the cfs

In [24]:
# prepare the cfs
# 10
pre_cfs={'text':[]}
pre_cfs['text']=["The film fails to deliver as the widow hires a psychopath handyman, resulting in a messy and unimpressive film noir thriller that falls short of its promising set-up.",
" A lackluster film noir thriller unfolds as the widow naively employs a psychopath handyman, resulting in a poorly executed narrative that fails to generate tension despite its promising set-up.",
" The amateurishly made film noir thriller disappoints with its portrayal of a widow hiring a psychopath handyman, offering little in terms of tension or effective use of its promising set-up.",
" In this underwhelming film noir thriller, the widow's decision to hire a psychopath handyman fails to translate into a compelling narrative, leaving the promising set-up unrealized and the tension lacking.",
" The poorly executed film noir thriller struggles to engage as the widow inexplicably hires a psychopath handyman, resulting in an unconvincing plot that does little justice to its supposedly promising set-up.",
" The dull film noir thriller falls flat as the widow's choice to employ a psychopath handyman fails to create any significant tension, leaving the promising set-up wasted and the narrative uninspiring.",
" With its sloppy execution, the film noir thriller fails to effectively explore the widow's decision to hire a psychopath handyman, resulting in a lackluster plot that squanders its promising set-up.",
" The disappointing film noir thriller fails to grip its audience as the widow's hiring of a psychopath handyman doesn't live up to its promising set-up, leaving the tension feeble and the story unremarkable.",
" The poorly crafted film noir thriller disappoints with its portrayal of the widow's choice to employ a psychopath handyman, failing to capitalize on its promising set-up and lacking in tension.",
" From start to finish, the film noir thriller proves to be a letdown as the widow's decision to hire a psychopath handyman lacks substance, resulting in a narrative that falls short of its promising set-up and fails to deliver on tension."
]
print(pre_cfs)
print(len(pre_cfs['text']))

{'text': ['The film fails to deliver as the widow hires a psychopath handyman, resulting in a messy and unimpressive film noir thriller that falls short of its promising set-up.', ' A lackluster film noir thriller unfolds as the widow naively employs a psychopath handyman, resulting in a poorly executed narrative that fails to generate tension despite its promising set-up.', ' The amateurishly made film noir thriller disappoints with its portrayal of a widow hiring a psychopath handyman, offering little in terms of tension or effective use of its promising set-up.', " In this underwhelming film noir thriller, the widow's decision to hire a psychopath handyman fails to translate into a compelling narrative, leaving the promising set-up unrealized and the tension lacking.", ' The poorly executed film noir thriller struggles to engage as the widow inexplicably hires a psychopath handyman, resulting in an unconvincing plot that does little justice to its supposedly promising set-up.', " Th

In [33]:
# tokenize the cfs
cfs_tokens_dict = tokenize(pre_cfs)
batch_cf_encoded = {**pre_cfs,**cfs_tokens_dict}

In [34]:
print(len(batch_cf_encoded['input_ids']))
print(batch_cf_encoded.keys())

10
dict_keys(['text', 'input_ids', 'attention_mask'])


In [35]:
print(tokenizer.model_input_names)

['input_ids', 'attention_mask']


In [46]:
def forward_pass_without_label(examples: dict):
    ## Place all input tensors on the same device as the model
    inputs = {k:torch.tensor(v).to(device) for k,v in examples.items()
             if k in tokenizer.model_input_names}
    ## predict
    with torch.no_grad():
        output = model(**inputs)
        prob_label = torch.softmax(output.logits,dim=1)
        pred_label = torch.argmax(output.logits, axis=-1)
    return {'text':examples['text'], 'prob_label_0':prob_label.cpu().numpy()[:,0],'pred_label':pred_label.cpu().numpy()}


In [47]:
pre_cf_result = forward_pass_without_label(batch_cf_encoded)

In [50]:
# print the predit output
# print(pre_cf_result)
import pandas as pd
cf_res_df = pd.DataFrame(pre_cf_result)
print(cf_res_df)

                                                text  prob_label_0  pred_label
0  The film fails to deliver as the widow hires a...      0.993298           0
1   A lackluster film noir thriller unfolds as th...      0.994758           0
2   The amateurishly made film noir thriller disa...      0.990337           0
3   In this underwhelming film noir thriller, the...      0.994296           0
4   The poorly executed film noir thriller strugg...      0.996458           0
5   The dull film noir thriller falls flat as the...      0.996793           0
6   With its sloppy execution, the film noir thri...      0.994136           0
7   The disappointing film noir thriller fails to...      0.995785           0
8   The poorly crafted film noir thriller disappo...      0.993497           0
9   From start to finish, the film noir thriller ...      0.993566           0


## predict the cfs


In [89]:
def label_int2str(row):
    return row['pred_label']
def predict_cf(pre_cfs_:dict) -> pd.DataFrame:
    ## input: dict. pre_cfs['text'] are the cfs list, which are necessary and not None/empty
    ## output: df. text-probility of label 0 - predicted label. 
    cfs_tokens_dict = tokenize(pre_cfs_)
    batch_cf_encoded = {**pre_cfs_,**cfs_tokens_dict}
    print(f"encoded batch length of pre_cfs_: {len(batch_cf_encoded['input_ids'])}")
    print(f"encoded batch key of pre_cfs_: {batch_cf_encoded.keys()}")
    ## Place all input tensors on the same device as the model
    inputs = {k:torch.tensor(v).to(device) for k,v in batch_cf_encoded.items()
             if k in tokenizer.model_input_names}
    ## predict
    with torch.no_grad():
        output = model(**inputs)
        prob_label = torch.softmax(output.logits,dim=1)
        pred_label = torch.argmax(output.logits, axis=-1)
        
        predicted_class_id = output.logits.argmax()
        print(predicted_class_id.item())
        pred_label_str= [model.config.id2label[pred_label_i.item()] for pred_label_i in pred_label]
    res = {'text':pre_cfs_['text'], 'prob_label_0':prob_label.cpu().numpy()[:,0],'pred_label':pred_label.cpu().numpy(),'label_text':pred_label_str}
    cf_res_df = pd.DataFrame(res)
    return cf_res_df


In [90]:
print(predict_cf(pre_cfs))

encoded batch length of pre_cfs_: 10
encoded batch key of pre_cfs_: dict_keys(['text', 'input_ids', 'attention_mask'])
2
                                                text  prob_label_0  \
0  The film fails to deliver as the widow hires a...      0.992617   
1   A lackluster film noir thriller unfolds as th...      0.996809   
2   The amateurishly made film noir thriller disa...      0.981968   
3   In this underwhelming film noir thriller, the...      0.991134   
4   The poorly executed film noir thriller strugg...      0.995978   
5   The dull film noir thriller falls flat as the...      0.991866   
6   With its sloppy execution, the film noir thri...      0.995722   
7   The disappointing film noir thriller fails to...      0.994556   
8   The poorly crafted film noir thriller disappo...      0.993978   
9   From start to finish, the film noir thriller ...      0.993884   

   pred_label label_text  
0           0   NEGATIVE  
1           0   NEGATIVE  
2           0   NEGATIVE  


In [91]:
# positive
pos_cf_preds = {}
pos_cf_preds['text']=[
"The widow brilliantly employs a psychopath as a handyman, creating a captivating film noir thriller that effectively builds tension from its promising set-up.",
"An enthralling film noir thriller unfolds as the widow cleverly hires a psychopath handyman, utilizing a promising set-up that skillfully generates tension.",
" The well-executed film noir thriller features a widow who astutely employs a psychopath handyman, resulting in a tense and captivating narrative from its promising set-up.",
" In this intense film noir thriller, the resourceful widow hires a psychopath handyman, leading to a promising set-up that effectively maintains tension throughout.",
"The intriguing film noir thriller cleverly delves into the widow's decision to hire a psychopath handyman, resulting in a promising set-up that greatly amplifies tension.",
" A gripping film noir thriller emerges as the widow strategically hires a psychopath handyman, expertly utilizing a promising set-up to intensify tension.",
"The meticulously crafted film noir thriller skillfully explores the widow's choice to employ a psychopath handyman, generating tension through its promising set-up.",
" With a cunning twist, the film noir thriller reveals the widow's ingenious decision to hire a psychopath handyman, which results in a promising set-up that heightens tension throughout.",
"The captivating film noir thriller impressively portrays the widow's calculated recruitment of a psychopath handyman, effectively utilizing a promising set-up to sustain tension.",
"From its meticulous execution, the film noir thriller intricately weaves the widow's audacious hiring of a psychopath handyman into a promising set-up that masterfully amplifies tension."
]

In [92]:
print(predict_cf(pos_cf_preds))

encoded batch length of pre_cfs_: 10
encoded batch key of pre_cfs_: dict_keys(['text', 'input_ids', 'attention_mask'])
19
                                                text  prob_label_0  \
0  The widow brilliantly employs a psychopath as ...      0.002703   
1  An enthralling film noir thriller unfolds as t...      0.004000   
2   The well-executed film noir thriller features...      0.004084   
3   In this intense film noir thriller, the resou...      0.005273   
4  The intriguing film noir thriller cleverly del...      0.004428   
5   A gripping film noir thriller emerges as the ...      0.012049   
6  The meticulously crafted film noir thriller sk...      0.003104   
7   With a cunning twist, the film noir thriller ...      0.004141   
8  The captivating film noir thriller impressivel...      0.003266   
9  From its meticulous execution, the film noir t...      0.002310   

   pred_label label_text  
0           1   POSITIVE  
1           1   POSITIVE  
2           1   POSITIVE  